# Legalis Legal Analysis Demo

This notebook demonstrates how to use the Legalis API Python SDK for legal analysis tasks.

## Setup

First, install the Legalis API client:

```bash
pip install -e ../crates/legalis-api/sdk/python
```

In [ ]:
import legalis_client
from legalis_client.api import statutes_api, verification_api, visualization_api
from legalis_client.model.create_statute_request import CreateStatuteRequest
from legalis_client.model.verify_request import VerifyRequest
import json
import pandas as pd
import matplotlib.pyplot as plt

# Configure API client
configuration = legalis_client.Configuration(
    host="http://localhost:3000"
)

api_client = legalis_client.ApiClient(configuration)
statutes = statutes_api.StatutesApi(api_client)
verification = verification_api.VerificationApi(api_client)
visualization = visualization_api.VisualizationApi(api_client)

## 1. Create a Legal Statute

Let's create a simple age restriction statute.

In [ ]:
statute_data = {
    "id": "age-restriction-demo",
    "name": "Age Restriction for Service Access",
    "description": "Users must be 18 years or older to access this service",
    "condition": {
        "type": "Age",
        "operator": "GreaterOrEqual",
        "value": 18
    },
    "effect": {
        "type": "Grant",
        "target": "service_access"
    }
}

request = CreateStatuteRequest(**statute_data)
response = statutes.create_statute(create_statute_request=request)
print(f"Statute created: {response}")

## 2. List All Statutes

Retrieve and display all statutes in the system.

In [ ]:
statute_list = statutes.list_statutes()

# Convert to DataFrame for easy viewing
df = pd.DataFrame([
    {
        'ID': s.id,
        'Name': s.name,
        'Description': s.description[:50] + '...' if len(s.description) > 50 else s.description
    }
    for s in statute_list.data
])

df

## 3. Verify a Statute

Check if a statute is logically consistent and free of contradictions.

In [ ]:
verify_request = VerifyRequest(
    statute_id="age-restriction-demo"
)

verification_result = verification.verify_statute(verify_request=verify_request)

print(f"Verification Result: {verification_result.valid}")
if verification_result.errors:
    print(f"Errors: {verification_result.errors}")
else:
    print("No errors found!")

## 4. Visualize Statute Logic

Generate a visual representation of the statute's decision logic.

In [ ]:
viz_response = visualization.visualize_statute(
    statute_id="age-restriction-demo",
    format="mermaid"
)

print("Mermaid Diagram:")
print(viz_response.content)

## 5. Batch Analysis

Analyze multiple statutes and compare their characteristics.

In [ ]:
# Get all statutes
all_statutes = statutes.list_statutes().data

# Analyze characteristics
analysis_data = []
for statute in all_statutes:
    result = verification.verify_statute(
        verify_request=VerifyRequest(statute_id=statute.id)
    )
    analysis_data.append({
        'Statute': statute.name,
        'Valid': result.valid,
        'Complexity': len(str(statute.condition)) if statute.condition else 0
    })

analysis_df = pd.DataFrame(analysis_data)
analysis_df

In [ ]:
# Visualize complexity distribution
plt.figure(figsize=(10, 6))
plt.bar(analysis_df['Statute'], analysis_df['Complexity'])
plt.xlabel('Statute')
plt.ylabel('Complexity Score')
plt.title('Statute Complexity Comparison')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 6. Advanced: Conflict Detection

Check for conflicts between multiple statutes.

In [ ]:
# Create two potentially conflicting statutes
statute1 = CreateStatuteRequest(
    id="rule-a",
    name="Rule A",
    description="Grant access if age >= 18",
    condition={"type": "Age", "operator": "GreaterOrEqual", "value": 18},
    effect={"type": "Grant", "target": "access"}
)

statute2 = CreateStatuteRequest(
    id="rule-b",
    name="Rule B",
    description="Deny access if age >= 18",
    condition={"type": "Age", "operator": "GreaterOrEqual", "value": 18},
    effect={"type": "Deny", "target": "access"}
)

statutes.create_statute(create_statute_request=statute1)
statutes.create_statute(create_statute_request=statute2)

# Verify for conflicts
conflict_check = verification.verify_statutes(
    verify_request=VerifyRequest(statute_ids=["rule-a", "rule-b"])
)

print(f"Conflict detected: {not conflict_check.valid}")
if conflict_check.errors:
    print(f"Conflicts: {conflict_check.errors}")

## Cleanup

Delete test statutes.

In [ ]:
for statute_id in ["age-restriction-demo", "rule-a", "rule-b"]:
    try:
        statutes.delete_statute(statute_id=statute_id)
        print(f"Deleted: {statute_id}")
    except Exception as e:
        print(f"Could not delete {statute_id}: {e}")

## Summary

This notebook demonstrated:

1. Creating legal statutes via the API
2. Listing and querying statutes
3. Formal verification of legal rules
4. Visualization of statute logic
5. Batch analysis of multiple statutes
6. Conflict detection between rules

For more information, visit: https://github.com/cool-japan/legalis